<a href="https://colab.research.google.com/github/kweyuchesa/WEEK-4---MORINGA-IP/blob/master/Moringa_Data_Science_Prep_W4_Independent_Project_2019_07_Kevin_Chesa_Python_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#libraries for our analysis 

In [0]:
## Import the packages and libraries
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import plotly as py
import seaborn as sns 
import os


Importing Our **Autolib_dataset.xlsx** Dataset for analysis and Exploration 

In [14]:
# Let's read the data from the CSV file and create the dataframe to be used
url = 'http://bit.ly/Autolibdataset'
url

'http://bit.ly/Autolibdataset'

#Creating an SQL Environment for Loading Huge Dataset

In [19]:
import os
import csv
import sqlite3
from sqlalchemy import create_engine

csv_database = create_engine('sqlite:////csv_database.db')
chunk_size = 1000000
i = 0
j = 0
for df in pd.read_csv(url, chunksize = chunk_size, iterator = True):
    df = df.rename(columns = {c: c.replace(' ','') for c in df.columns})
    df.index += j

df.to_sql('auto', csv_database, if_exists = 'append' )
j = df.index[-1]+1

print(' | index: {}'.format(j))

 | index: 14421829


In [21]:
#Finding the size of the dataset 
auto.tail()

,index,Unnamed:0,Address,Cars,Bluecarcounter,Utilibcounter,Utilib1.4counter,ChargeSlots,ChargingStatus,City,Displayedcomment,ID,Kind,Geopoint,Postalcode,Publicname,Rentalstatus,Scheduledat,Slots,Stationtype,Status,Subscriptionstatus,year,month,day,hour,minute
460527,14421817,234,50 Boulevard Auguste Blanqui,3,3,0,0,0,nonexistent,Paris,None,paris-augusteblanqui-50,STATION,"48.8299453, 2.3499186",75013,Paris/Auguste Blanqui/50,operational,None,1,station,ok,nonexistent,2018,4,9,23,59
460528,14421819,372,4 rue Turgot,0,0,0,0,0,broken,Paris,None,paris-turgot-4,STATION,"48.8800786, 2.3455235",75009,Paris/Turgot/4,broken,None,0,station,closed,nonexistent,2018,4,9,23,59
460529,14421821,605,125 bis avenue Parmentier,0,0,0,0,1,operational,Paris,None,paris-parmentier-125,STATION,"48.8690795, 2.371679",75011,Paris/Parmentier/125,operational,None,5,station,ok,nonexistent,2018,4,9,23,59
460530,14421822,634,8 rue Danton,0,0,0,0,2,operational,Paris,None,paris-danton-8,STATION,"48.85262, 2.342107",75006,Paris/Danton/8,operational,None,5,station,ok,nonexistent,2018,4,9,23,59
460531,14421826,996,3 rue Olivier de Serres,5,5,0,0,0,operational,Paris,None,paris-olivierdeserres-3,STATION,"48.8370178, 2.2986736",75015,Paris/Olivier de Serres/3,operational,None,1,station,ok,nonexistent,2018,4,9,23,59


#Retreaving csv file into our sql lite environment 

DESCRIBING THE DATASET

In [0]:
#retreaving csv file into our sql lite environment 
auto = pd.read_sql_query('SELECT * from auto WHERE city == "Paris" ', csv_database, )

In [0]:
auto.to_csv('auto_clean.csv')

In [0]:
auto = pd.read_csv('auto_clean.csv', dtype = {'year':str,'month':str,'day':str,'hour':str,'minute':str })

In [0]:
auto['date'] = auto.year.str.cat([auto.month,auto.day], sep = '/')

In [0]:
auto['time'] = auto.hour.str.cat([auto.minute], sep = ':')

In [0]:
auto['date_time'] = auto.date.str.cat([auto.time], sep = ' ')

In [0]:
auto.drop(['year','month', 'day', 'minute',  'date', 'time'],axis = 1, inplace = True)

In [0]:
auto.to_csv('auto_clean.csv')

In [0]:
auto.head()

Renaming Our dataset

Preview our dataset

In [92]:
auto.tail()

,Unnamed: 0,Bluecarcounter,Utilibcounter,Utilib1.4counter,City,Postalcode,Publicname,Stationtype,Status,Subscriptionstatus,hour,date_time
690793,690793,3,0,0,Paris,75013,Paris/Auguste Blanqui/50,station,ok,nonexistent,23,2018/4/9 23:59
690794,690794,0,0,0,Paris,75009,Paris/Turgot/4,station,closed,nonexistent,23,2018/4/9 23:59
690795,690795,0,0,0,Paris,75011,Paris/Parmentier/125,station,ok,nonexistent,23,2018/4/9 23:59
690796,690796,0,0,0,Paris,75006,Paris/Danton/8,station,ok,nonexistent,23,2018/4/9 23:59
690797,690797,5,0,0,Paris,75015,Paris/Olivier de Serres/3,station,ok,nonexistent,23,2018/4/9 23:59


#Validity ~ Dropping Unnecesary columns/Variables 

In [0]:
auto.drop(['Unnamed: 0', 'Unnamed:0', 'index',  'Displayedcomment', 'ID', 'Geopoint', 
           'Scheduledat','Address','ChargeSlots', 'ChargingStatus','Kind', 'Rentalstatus','Slots',],axis=1, inplace=True)

In [77]:
# checking the columns types 
auto.dtypes

Bluecarcounter         int64
Utilibcounter          int64
Utilib1.4counter       int64
City                  object
Postalcode             int64
Publicname            object
Stationtype           object
Status                object
Subscriptionstatus    object
hour                  object
date_time             object
dtype: object

#Completeness ~ Null Values Assessment 

In [78]:
#Checking for null values  
auto.isnull().sum().sum()

0

In [0]:
#preview our dataset to ascertain its dropped columns 
auto.head()

#Duplicates 

In [0]:
#Duplicates and Drop them ~ Scheduled at has the most duplicates 14310213  
auto[auto.duplicated(subset = None)]

In [36]:
#Checking if its only city we are dealing with 
auto.City.unique()

array(['Paris'], dtype=object)

#Uniformity ~ Date Type Conversion 

In [0]:
#Uniformity - datetypes as strings 
auto = pd.read_csv('auto_clean.csv', dtype = {'year':str,'month':str,'day':str,'hour':str,'minute':str })

In [0]:
#converting  date_time column 
auto['date_time'] = pd.to_datetime( auto['date_time'])

Displaying Clean Dataset

In [94]:
#DISPLAYING OUR CLEAN DATASET
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690798 entries, 0 to 690797
Data columns (total 12 columns):
Unnamed: 0            690798 non-null int64
Bluecarcounter        690798 non-null int64
Utilibcounter         690798 non-null int64
Utilib1.4counter      690798 non-null int64
City                  690798 non-null object
Postalcode            690798 non-null int64
Publicname            690798 non-null object
Stationtype           690798 non-null object
Status                690798 non-null object
Subscriptionstatus    690798 non-null object
hour                  690798 non-null object
date_time             690798 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 63.2+ MB


Saving our clean dataset as csv

In [0]:
#SAVING THE CLEAN DATASET AS CSV
auto.to_csv('auto_clean.csv')

In [0]:
auto.count()

#ANALYSIS ~ Answering Research Questions 

1. What was the most popular hour of the day for picking up a shared electric car (Bluecar) 
in the city of Paris over the month of April 2018?

In [0]:
#ensuring the city is Paris 
auto[auto.City == 'Paris'] 

In [0]:
#this lists the hours for picking the cars 
auto = auto[auto['Bluecarcounter']==auto['Bluecarcounter'].min().head()]

2.What is the most popular postal code per hour for returning blue cars? 78005 most popular with 14419 units of cars 

In [83]:
auto.groupby(['Postalcode',auto.date_time.dt.hour,'Bluecarcounter'])['Bluecarcounter'].count().sort_values(ascending = False).head(20)

AttributeError: ignored

3. Most Popular Station 

In [0]:
dff.groupby(['public_name','cars']).count().sort_values(ascending = False).head(500)

4. Utilib_counter Station at the most popular picking hour for blue cars 

In [0]:
dff.groupby('public_name',dff.date.dt.hour.['utilib_counter'],['utilib_counter'].count().sort_values(ascending = False)

5. 